In [19]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

df = pd.read_csv("all_subs_windowed.csv")

df['label'] = df['label'] - 1   # convert labels (1-4 to 0-3)
df.head()  # 1203 rows full data

,subject,label,S1_D1_w1,S1_D1_w2,S1_D1_w3,S1_D2_w1,S1_D2_w2,S1_D2_w3,S2_D1_w1,S2_D1_w2,...,S9_D6_w3,S9_D8_w1,S9_D8_w2,S9_D8_w3,S10_D7_w1,S10_D7_w2,S10_D7_w3,S10_D8_w1,S10_D8_w2,S10_D8_w3
0,sub-101,2,-7.120779e-08,-1.591482e-08,4.037125e-08,1.059506e-07,-1.591482e-08,5.576864e-09,-5.745610e-08,1.158969e-09,...,-6.338075e-08,4.680183e-08,-6.375286e-08,1.427296e-08,4.680183e-08,-7.726111e-09,1.427296e-08,-7.726111e-09,1.427296e-08,-7.726111e-09
1,sub-101,0,-7.105098e-08,-1.428847e-07,-1.661490e-07,-1.067971e-07,-1.428847e-07,-1.549473e-07,-1.100738e-07,-9.783376e-08,...,-9.825674e-08,-1.156452e-07,-1.293089e-07,-1.468958e-07,-1.156452e-07,-1.101812e-07,-1.468958e-07,-1.101812e-07,-1.468958e-07,-1.101812e-07
2,sub-101,3,6.854295e-08,1.563502e-07,1.207582e-07,1.066198e-07,1.563502e-07,1.385606e-07,1.212980e-07,4.591666e-08,...,4.139978e-08,2.738268e-08,7.855582e-08,1.718898e-08,2.738268e-08,4.968935e-09,1.718898e-08,4.968935e-09,1.718898e-08,4.968935e-09
3,sub-101,1,1.206235e-07,1.431428e-07,2.246828e-08,2.433219e-08,1.431428e-07,1.236011e-07,1.035702e-07,1.868319e-09,...,-1.115965e-07,-2.759947e-08,-1.808735e-07,5.235214e-08,-2.759947e-08,1.151776e-08,5.235214e-08,1.151776e-08,5.235214e-08,1.151776e-08
4,sub-101,2,-1.319688e-07,-1.376738e-07,-2.001670e-07,-1.707070e-07,-1.376738e-07,-6.784068e-08,-1.378276e-07,-6.926143e-08,...,-1.588279e-07,-1.109203e-07,-2.869832e-07,-1.373283e-07,-1.109203e-07,-1.356308e-07,-1.373283e-07,-1.356308e-07,-1.373283e-07,-1.356308e-07


## LOSO CV - base models

### XGBoost

In [4]:
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

# Features, label, and subject group
X = df.drop(columns=['label', 'subject'])
y = df['label']
groups = df['subject']  # <-- subject strings like 'sub-133'

logo = LeaveOneGroupOut()
accuracies = [] 

for train_idx, test_idx in logo.split(X, y, groups=groups):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Optional, scale inside loop to prevent leakage
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Simple classifier
    model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)

    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

print(f"Mean Accuracy (LOSO): {sum(accuracies)/len(accuracies):.4f}")

C:\Users\humag\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:46:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\humag\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:46:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\humag\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:46:25] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are

✅ Mean Accuracy (LOSO): 0.2635


### LDA

In [7]:
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Features, label, and subject group
X = df.drop(columns=['label', 'subject'])
y = df['label']
groups = df['subject'] 

logo = LeaveOneGroupOut()
accuracies = []

for train_idx, test_idx in logo.split(X, y, groups=groups):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Scale inside loop to avoid leakage
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # LDA model
    model = LinearDiscriminantAnalysis()
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)

    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

print(f"Mean Accuracy (LOSO with LDA): {sum(accuracies)/len(accuracies):.4f}")


✅ Mean Accuracy (LOSO with LDA): 0.2585


### RF

In [9]:
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

# Features, label, and subject group
X = df.drop(columns=['label', 'subject'])
y = df['label']
groups = df['subject']  # <-- subject strings like 'sub-133'

logo = LeaveOneGroupOut()
accuracies = []

for train_idx, test_idx in logo.split(X, y, groups=groups):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Optional: scale inside loop (not critical for RF, but harmless)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Random Forest Classifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)

    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

print(f"Mean Accuracy (LOSO with Random Forest): {sum(accuracies)/len(accuracies):.4f}")

✅ Mean Accuracy (LOSO with Random Forest): 0.2989


### LGBM

In [10]:
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier

# Features, label, and subject group
X = df.drop(columns=['label', 'subject'])
y = df['label']
groups = df['subject'] 

logo = LeaveOneGroupOut()
accuracies = []

for train_idx, test_idx in logo.split(X, y, groups=groups):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # LGBM Classifier
    model = LGBMClassifier(random_state=42)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)

    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

print(f"Mean Accuracy (LOSO with LGBM): {sum(accuracies)/len(accuracies):.4f}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002336 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18360
[LightGBM] [Info] Number of data points in the train set: 1143, number of used features: 72
[LightGBM] [Info] Start training from score -1.388922
[LightGBM] [Info] Start training from score -1.385420
[LightGBM] [Info] Start training from score -1.385420
[LightGBM] [Info] Start training from score -1.385420
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002595 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18360
[LightGBM] [Info] Number of data points in the train set: 1143, number of used features: 72
[LightGBM] [Info] Start training from score -1.388922
[LightGBM] [Info] Start training from score -1.385420
[LightGBM] [Info] Start training from score -1.385420
[LightGBM] [Info] Start training from score 

#### LGBM Optuna

In [14]:
import optuna
import numpy as np
import pandas as pd
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from sklearn.pipeline import make_pipeline

X = df.drop(columns=['subject', 'label'])
y = df['label']
groups = df['subject']

# Define Optuna Objective
def objective(trial):
    # Suggested hyperparameters
    params = {
        'objective': 'multiclass',
        'num_class': 4, 
        'metric': 'multi_logloss',
        'verbosity': -1,
        'n_jobs': -1,

        # parameters that will be tuned
        'n_estimators': trial.suggest_int('n_estimators', 100, 700),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'num_leaves': trial.suggest_int('num_leaves', 16, 128),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'max_bin': trial.suggest_int('max_bin', 64, 255),
    }


    # Pipeline with scaling + LGBM
    clf = make_pipeline(
        StandardScaler(),
        LGBMClassifier(**params)
    )

    logo = LeaveOneGroupOut()
    scores = cross_val_score(clf, X, y, cv=logo.split(X, y, groups=groups), scoring="accuracy")
    return scores.mean()

# Run Optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Show best hyperparameters
print("✅ Best Hyperparameters:")
print(study.best_trial.params)

[I 2025-07-28 16:26:11,648] A new study created in memory with name: no-name-8b9fc2b8-eed7-481b-b4c0-e8d179386df6
[I 2025-07-28 16:27:07,512] Trial 0 finished with value: 0.25361150070126237 and parameters: {'n_estimators': 346, 'max_depth': 11, 'num_leaves': 43, 'learning_rate': 0.03560960604512054, 'feature_fraction': 0.8537768901900573, 'bagging_fraction': 0.6460194375053462, 'lambda_l1': 3.1174346573478506, 'lambda_l2': 3.529224410283165, 'min_child_samples': 32, 'max_bin': 167}. Best is trial 0 with value: 0.25361150070126237.
[I 2025-07-28 16:29:16,429] Trial 1 finished with value: 0.2519985974754558 and parameters: {'n_estimators': 661, 'max_depth': 7, 'num_leaves': 54, 'learning_rate': 0.01775111955036761, 'feature_fraction': 0.6680715587737244, 'bagging_fraction': 0.8625961613006219, 'lambda_l1': 0.10862589866372785, 'lambda_l2': 4.099853884835902, 'min_child_samples': 22, 'max_bin': 140}. Best is trial 0 with value: 0.25361150070126237.
[I 2025-07-28 16:29:42,140] Trial 2 fin

✅ Best Hyperparameters:
{'n_estimators': 158, 'max_depth': 14, 'num_leaves': 120, 'learning_rate': 0.0023129559233807115, 'feature_fraction': 0.8181304692531582, 'bagging_fraction': 0.9302593262691019, 'lambda_l1': 4.583363901282219, 'lambda_l2': 1.9891249768767543, 'min_child_samples': 43, 'max_bin': 248}


In [15]:
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
import numpy as np

# Run LOSO with Best Params
best_params = study.best_trial.params
best_params.update({
    'objective': 'multiclass',
    'num_class': 4,
   # 'metric': 'multi_logloss',
    
    'n_jobs': -1
})

logo = LeaveOneGroupOut()
test_accuracies = []
train_accuracies = []
subject_ids = []

for train_idx, test_idx in logo.split(X, y, groups=groups):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Save test subject
    test_subject = groups.iloc[test_idx].iloc[0]
    subject_ids.append(test_subject)

    # Scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Model training
    model = LGBMClassifier(**best_params)
    model.fit(X_train_scaled, y_train)

    # Predictions
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)

    # Accuracies
    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)

    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)

    print(f"📁 Test Subject: {test_subject} | 🟢 Train Acc: {train_acc:.2f} | 🔵 Test Acc: {test_acc:.2f}")

# Summary
mean_train = np.mean(train_accuracies)
mean_test = np.mean(test_accuracies)

print(f"\n✅ Mean Train Accuracy: {mean_train:.2f}")
print(f"✅ Mean Test Accuracy (LOSO): {mean_test:.3f}")

📁 Test Subject: sub-101 | 🟢 Train Acc: 0.66 | 🔵 Test Acc: 0.28
📁 Test Subject: sub-105 | 🟢 Train Acc: 0.65 | 🔵 Test Acc: 0.30
📁 Test Subject: sub-107 | 🟢 Train Acc: 0.63 | 🔵 Test Acc: 0.33
📁 Test Subject: sub-108 | 🟢 Train Acc: 0.63 | 🔵 Test Acc: 0.28
📁 Test Subject: sub-109 | 🟢 Train Acc: 0.62 | 🔵 Test Acc: 0.30
📁 Test Subject: sub-112 | 🟢 Train Acc: 0.65 | 🔵 Test Acc: 0.28
📁 Test Subject: sub-113 | 🟢 Train Acc: 0.64 | 🔵 Test Acc: 0.23
📁 Test Subject: sub-119 | 🟢 Train Acc: 0.64 | 🔵 Test Acc: 0.33
📁 Test Subject: sub-120 | 🟢 Train Acc: 0.63 | 🔵 Test Acc: 0.25
📁 Test Subject: sub-121 | 🟢 Train Acc: 0.63 | 🔵 Test Acc: 0.38
📁 Test Subject: sub-123 | 🟢 Train Acc: 0.61 | 🔵 Test Acc: 0.25
📁 Test Subject: sub-124 | 🟢 Train Acc: 0.62 | 🔵 Test Acc: 0.38
📁 Test Subject: sub-125 | 🟢 Train Acc: 0.64 | 🔵 Test Acc: 0.42
📁 Test Subject: sub-126 | 🟢 Train Acc: 0.65 | 🔵 Test Acc: 0.22
📁 Test Subject: sub-127 | 🟢 Train Acc: 0.64 | 🔵 Test Acc: 0.30
📁 Test Subject: sub-129 | 🟢 Train Acc: 0.62 | 🔵 Test Ac

## Exclude sub 125 for testing web application

In [16]:
df = pd.read_csv("all_subs_windowed.csv")

df['label'] = df['label'] - 1

df = df[df['subject'] != 'sub-125']  # exclude subject 125
df.head()

,subject,label,S1_D1_w1,S1_D1_w2,S1_D1_w3,S1_D2_w1,S1_D2_w2,S1_D2_w3,S2_D1_w1,S2_D1_w2,...,S9_D6_w3,S9_D8_w1,S9_D8_w2,S9_D8_w3,S10_D7_w1,S10_D7_w2,S10_D7_w3,S10_D8_w1,S10_D8_w2,S10_D8_w3
0,sub-101,2,-7.120779e-08,-1.591482e-08,4.037125e-08,1.059506e-07,-1.591482e-08,5.576864e-09,-5.745610e-08,1.158969e-09,...,-6.338075e-08,4.680183e-08,-6.375286e-08,1.427296e-08,4.680183e-08,-7.726111e-09,1.427296e-08,-7.726111e-09,1.427296e-08,-7.726111e-09
1,sub-101,0,-7.105098e-08,-1.428847e-07,-1.661490e-07,-1.067971e-07,-1.428847e-07,-1.549473e-07,-1.100738e-07,-9.783376e-08,...,-9.825674e-08,-1.156452e-07,-1.293089e-07,-1.468958e-07,-1.156452e-07,-1.101812e-07,-1.468958e-07,-1.101812e-07,-1.468958e-07,-1.101812e-07
2,sub-101,3,6.854295e-08,1.563502e-07,1.207582e-07,1.066198e-07,1.563502e-07,1.385606e-07,1.212980e-07,4.591666e-08,...,4.139978e-08,2.738268e-08,7.855582e-08,1.718898e-08,2.738268e-08,4.968935e-09,1.718898e-08,4.968935e-09,1.718898e-08,4.968935e-09
3,sub-101,1,1.206235e-07,1.431428e-07,2.246828e-08,2.433219e-08,1.431428e-07,1.236011e-07,1.035702e-07,1.868319e-09,...,-1.115965e-07,-2.759947e-08,-1.808735e-07,5.235214e-08,-2.759947e-08,1.151776e-08,5.235214e-08,1.151776e-08,5.235214e-08,1.151776e-08
4,sub-101,2,-1.319688e-07,-1.376738e-07,-2.001670e-07,-1.707070e-07,-1.376738e-07,-6.784068e-08,-1.378276e-07,-6.926143e-08,...,-1.588279e-07,-1.109203e-07,-2.869832e-07,-1.373283e-07,-1.109203e-07,-1.356308e-07,-1.373283e-07,-1.356308e-07,-1.373283e-07,-1.356308e-07


In [17]:
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
import numpy as np

X = df.drop(columns=['subject', 'label'])
y = df['label']
groups = df['subject']

# Run LOSO with Best Params, the same parameters from training with all data
best_params = {'n_estimators': 158, 'max_depth': 14, 'num_leaves': 120, 'learning_rate': 0.0023129559233807115, 
                'feature_fraction': 0.8181304692531582, 'bagging_fraction': 0.9302593262691019, 
                'lambda_l1': 4.583363901282219, 'lambda_l2': 1.9891249768767543, 'min_child_samples': 43, 
                'max_bin': 248, 'objective': 'multiclass', 'num_class': 4, 
               # 'metric': 'multi_logloss', 
               'n_estimators': 100, 'n_jobs': -1}

logo = LeaveOneGroupOut()
test_accuracies = []
train_accuracies = []
subject_ids = []

for train_idx, test_idx in logo.split(X, y, groups=groups):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Save test subject
    test_subject = groups.iloc[test_idx].iloc[0]
    subject_ids.append(test_subject)

    # Scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Model training
    model = LGBMClassifier(**best_params)
    model.fit(X_train_scaled, y_train)

    # Predictions
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)

    # Accuracies
    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)

    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)

    print(f"📁 Test Subject: {test_subject} | 🟢 Train Acc: {train_acc:.2f} | 🔵 Test Acc: {test_acc:.2f}")

# Summary
mean_train = np.mean(train_accuracies)
mean_test = np.mean(test_accuracies)

print(f"\n✅ Mean Train Accuracy: {mean_train:.2f}")
print(f"✅ Mean Test Accuracy (LOSO): {mean_test:.3f}")

📁 Test Subject: sub-101 | 🟢 Train Acc: 0.61 | 🔵 Test Acc: 0.30
📁 Test Subject: sub-105 | 🟢 Train Acc: 0.63 | 🔵 Test Acc: 0.33
📁 Test Subject: sub-107 | 🟢 Train Acc: 0.62 | 🔵 Test Acc: 0.28
📁 Test Subject: sub-108 | 🟢 Train Acc: 0.63 | 🔵 Test Acc: 0.35
📁 Test Subject: sub-109 | 🟢 Train Acc: 0.63 | 🔵 Test Acc: 0.25
📁 Test Subject: sub-112 | 🟢 Train Acc: 0.63 | 🔵 Test Acc: 0.33
📁 Test Subject: sub-113 | 🟢 Train Acc: 0.63 | 🔵 Test Acc: 0.23
📁 Test Subject: sub-119 | 🟢 Train Acc: 0.60 | 🔵 Test Acc: 0.28
📁 Test Subject: sub-120 | 🟢 Train Acc: 0.62 | 🔵 Test Acc: 0.25
📁 Test Subject: sub-121 | 🟢 Train Acc: 0.62 | 🔵 Test Acc: 0.25
📁 Test Subject: sub-123 | 🟢 Train Acc: 0.60 | 🔵 Test Acc: 0.28
📁 Test Subject: sub-124 | 🟢 Train Acc: 0.61 | 🔵 Test Acc: 0.38
📁 Test Subject: sub-126 | 🟢 Train Acc: 0.63 | 🔵 Test Acc: 0.22
📁 Test Subject: sub-127 | 🟢 Train Acc: 0.63 | 🔵 Test Acc: 0.28
📁 Test Subject: sub-129 | 🟢 Train Acc: 0.60 | 🔵 Test Acc: 0.25
📁 Test Subject: sub-130 | 🟢 Train Acc: 0.61 | 🔵 Test Ac

In [18]:
# save model for app test

model.booster_.save_model("4Class_best_lgbm_loso_029.txt")

np.save("scaler2_mean.npy", scaler.mean_)
np.save("scaler2_scale.npy", scaler.scale_)